# 🔄 YOLO Model Retraining - Clean Dataset

## Purpose
After removing duplicate models and clearing directories, retrain YOLO model with fresh, clean dataset.

**Status**: All duplicate files removed, directories cleared ✅
**Goal**: Create clean trained model for production

---

## Pipeline
1. Import & Setup
2. Verify Dataset
3. Train YOLO Model
4. Validate Results
5. Save & Export Model
6. Compare with Previous Model (if backup exists)

In [ ]:
# 1️⃣ IMPORT & SETUP
import os
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# YOLO
from ultralytics import YOLO

# Utilities
import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

print("✅ Libraries imported successfully")

In [ ]:
# 2️⃣ VERIFY DATASET
PROJECT_ROOT = '/Users/miftahhadiyannoor/Documents/logistics-rag'
DATASET_PATH = os.path.join(PROJECT_ROOT, 'notebooks/dataset')
DATASET_CONFIG = os.path.join(DATASET_PATH, 'data.yaml')

print("📊 Dataset Verification")
print("="*60)

# Check dataset exists
if os.path.exists(DATASET_CONFIG):
    print(f"✅ Dataset config found: {DATASET_CONFIG}")
    
    # Read YAML
    import yaml
    with open(DATASET_CONFIG, 'r') as f:
        dataset_info = yaml.safe_load(f)
    
    print(f"\n📁 Dataset structure:")
    for key, value in dataset_info.items():
        if key in ['train', 'val', 'test']:
            path = os.path.join(DATASET_PATH, value)
            exists = os.path.exists(path)
            status = "✅" if exists else "❌"
            count = len(os.listdir(path)) if exists else 0
            print(f"  {status} {key}: {count} items")
    
    print(f"\n📋 Classes: {dataset_info.get('names', {})}")
else:
    print(f"❌ Dataset config not found: {DATASET_CONFIG}")
    print(f"   Please ensure data.yaml exists in {DATASET_PATH}")

In [ ]:
# 3️⃣ LOAD PRETRAINED YOLO & PREPARE FOR TRAINING
print("\n🤖 Loading YOLO Base Model")
print("="*60)

# Load a pretrained YOLOv8 model (small for faster training)
model = YOLO('yolov8n.pt')
print(f"✅ Base model loaded: yolov8n")
print(f"   Model parameters: {sum(p.numel() for p in model.model.parameters() if p.requires_grad):,}")

In [ ]:
# 4️⃣ TRAIN YOLO MODEL ON CLEAN DATASET
print("\n🚀 Starting YOLO Training")
print("="*60)

# Training configuration
EPOCHS = 100
IMG_SIZE = 640
BATCH_SIZE = 16
PATIENCE = 20  # Early stopping

print(f"📋 Training Config:")
print(f"   Epochs: {EPOCHS}")
print(f"   Image Size: {IMG_SIZE}")
print(f"   Batch Size: {BATCH_SIZE}")
print(f"   Early Stopping Patience: {PATIENCE}")
print(f"   Dataset: {DATASET_CONFIG}")

# Train the model
results = model.train(
    data=DATASET_CONFIG,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    patience=PATIENCE,
    device=0,  # GPU device 0
    save=True,
    project='runs/detect',
    name='train_clean',  # New clean training run
    exist_ok=False,  # Don't overwrite
    verbose=True,
    plots=True  # Generate plots
)

print(f"\n✅ Training complete!")
print(f"   Results saved to: runs/detect/train_clean/")

In [ ]:
# 5️⃣ VALIDATE MODEL ON TEST SET
print("\n🔍 Model Validation")
print("="*60)

# Load best model
best_model = YOLO('runs/detect/train_clean/weights/best.pt')

# Validate on validation set
val_results = best_model.val()

print(f"\n✅ Validation Metrics:")
print(f"   mAP50: {val_results.box.map50:.3f}")
print(f"   mAP50-95: {val_results.box.map:.3f}")
print(f"   Precision: {val_results.box.mp:.3f}")
print(f"   Recall: {val_results.box.mr:.3f}")

In [ ]:
# 6️⃣ DISPLAY TRAINING RESULTS
print("\n📈 Training Results Visualization")
print("="*60)

# Paths to result images
results_dir = 'runs/detect/train_clean'
result_images = [
    'results.png',
    'confusion_matrix.png',
    'confusion_matrix_normalized.png'
]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
axes = axes.flatten()

for idx, img_name in enumerate(result_images):
    img_path = os.path.join(results_dir, img_name)
    if os.path.exists(img_path):
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        axes[idx].imshow(img_rgb)
        axes[idx].set_title(img_name.replace('.png', '').replace('_', ' ').title())
        axes[idx].axis('off')
    else:
        axes[idx].text(0.5, 0.5, f'Not found: {img_name}', 
                      ha='center', va='center')
        axes[idx].set_title(img_name)

plt.tight_layout()
plt.show()

print("✅ Results displayed")

In [ ]:
# 7️⃣ EXPORT TRAINED MODEL
print("\n💾 Exporting Trained Model")
print("="*60)

# Copy best model to production location
import shutil

SOURCE_MODEL = 'runs/detect/train_clean/weights/best.pt'
EXPORT_LOCATIONS = [
    'production_api/model/best.pt',
    'deployment/models/best.pt',
    'yolov8-container-inspection/best.pt'
]

for dest in EXPORT_LOCATIONS:
    os.makedirs(os.path.dirname(dest), exist_ok=True)
    shutil.copy(SOURCE_MODEL, dest)
    print(f"  ✅ Model copied to: {dest}")

print(f"\n✅ Model exported to all locations!")

# Get model size
model_size = os.path.getsize(SOURCE_MODEL) / (1024*1024)
print(f"   Model size: {model_size:.2f}MB")

In [ ]:
# 8️⃣ TEST MODEL ON SAMPLE IMAGE
print("\n🧪 Testing Model on Sample")
print("="*60)

# Load trained model
test_model = YOLO('runs/detect/train_clean/weights/best.pt')

# Find a sample image from validation set
val_images_dir = os.path.join(DATASET_PATH, 'images/val')

if os.path.exists(val_images_dir):
    images = [f for f in os.listdir(val_images_dir) if f.endswith(('.jpg', '.png'))]
    
    if images:
        # Test on first image
        test_image_path = os.path.join(val_images_dir, images[0])
        print(f"   Testing on: {images[0]}")
        
        # Run inference
        results = test_model.predict(test_image_path)
        
        # Display results
        result = results[0]
        annotated_frame = result.plot()
        
        # Show
        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB))
        plt.title(f"Detection Results: {len(result.boxes)} objects detected")
        plt.axis('off')
        plt.show()
        
        print(f"\n✅ Detections: {len(result.boxes)} objects")
        
        # Show class distribution
        if len(result.boxes) > 0:
            print(f"\n   Classes detected:")
            for cls_id, cls_name in dataset_info['names'].items():
                count = (result.boxes.cls == cls_id).sum().item()
                if count > 0:
                    print(f"     - {cls_name}: {count}")
    else:
        print(f"❌ No validation images found in {val_images_dir}")
else:
    print(f"❌ Validation directory not found: {val_images_dir}")

In [ ]:
# 9️⃣ SUMMARY & NEXT STEPS
print("\n" + "="*60)
print("✅ YOLO RETRAINING COMPLETE")
print("="*60)

print(f"\n📊 Summary:")
print(f"   ✅ Trained on clean dataset")
print(f"   ✅ Model validated")
print(f"   ✅ Model exported to production locations")
print(f"   ✅ Sample inference tested")

print(f"\n📁 Output Locations:")
print(f"   Training results: runs/detect/train_clean/")
print(f"   Best model: runs/detect/train_clean/weights/best.pt")
print(f"   Validation plots: runs/detect/train_clean/")

print(f"\n🚀 Next Steps:")
print(f"   1. Regenerate vector database (07_VECTOR_DB_REGEN.ipynb)")
print(f"   2. Test production pipeline")
print(f"   3. Monitor model performance in production")

print(f"\n✨ Status: Ready for production!")
print("="*60)